In [ ]:
import json
from snowflake.snowpark.session import Session
from snowflake.snowpark.functions import *
from snowflake.snowpark.types import *
from config import snowflake_conn_prop
import pandas as pd

session = Session.builder.configs(snowflake_conn_prop).create()

In [ ]:
# how I would do in pandas
df = pd.read_csv('./articles.csv')
df.head(5)

In [ ]:
session.file.put('./articles.csv', '@~/', overwrite=True)
schema = StructType([
  StructField("author", StringType()),
  StructField("claps", StringType()),
  StructField("reading_time", IntegerType()),
  StructField("link", StringType()),
  StructField("title", StringType()),
  StructField("text", StringType())
])
df = session.read.option("skip_header", 1).option("field_delimiter", ",").option("FIELD_OPTIONALLY_ENCLOSED_BY", "\"").schema(schema).csv('@~/articles.csv')
df_pd = df.toPandas()
df_pd.head(5)

In [ ]:
# if I wanted to write this to a table (which I thought I had to do to run an external function)
# df.write.mode('overwrite').save_as_table('articles')

In [ ]:
df_nonsql = df.limit(1).select(
    *df.columns, call_builtin('my_external_function', df.text).alias("TL_DR")
    )
df_nonsql.show()